In [1]:
import requests
import bs4
import re
import random
import time
import pandas as pd
import csv

In [2]:
with open('titlelinks_2007to2017.txt', 'r') as f:
  reader = csv.reader(f)
  title_sublink_2007to2017 = list(reader)

In [3]:
movie_link =[]
for link in title_sublink_2007to2017:
    url = 'http://www.imdb.com'+ str(link[0])
    movie_link.append(url)
    
# movie_link

In [4]:
movie_list = []
movie_link_test = movie_link
for url_m in movie_link_test:
    movie_detial = []
    mtime = None
    color = None
    language = None 
    budget = None
    gross = None
    facebook_link = None
    offcialsite = None
    
    #get the soup of each movie
    msoup = bs4.BeautifulSoup(requests.get(url_m).text, 'html5lib')
    m_main = msoup.find('div', attrs = {'id':'main_top'})
    # get the title of the movie
    title = m_main.find('h1', attrs={'itemprop':'name'}).text.strip()
    subtext = m_main.find_all('div', attrs={'class':'subtext'})
    #movie content rating
    if subtext[0].find('meta', attrs={'itemprop':'contentRating'}):
        moviveclass = subtext[0].find('meta', attrs={'itemprop':'contentRating'})['content']
    else:
        moviveclass=None
    # movie genre, there are several genre at the same time
    genre_list = subtext[0].find_all('span', attrs={'itemprop':'genre'})
    genre = [g.text.strip() for g in genre_list]
    # date published
    if subtext[0].find('meta', attrs={'itemprop':'datePublished'}):
        datepublish = subtext[0].find('meta', attrs={'itemprop':'datePublished'})['content']
    else:
        datepublish = None
    #get the rating of the movie
    rate = m_main.find('span', attrs={'itemprop':'ratingValue'}).text.strip()
    # rating count
    ratingCount = m_main.find('span', attrs={'itemprop':'ratingCount'}).text.strip()
    # get the dirctor(s)
    directorSpan = m_main.find_all('span', attrs={'itemprop':'director'})
    director = [d.text.strip() for d in directorSpan]
    # get the writer(s)
    writerSpan = m_main.find_all('span', attrs={'itemprop':'creator'})
    writer = [w.text.strip() for w in writerSpan]
    #get the actors
    actorSpan = m_main.find_all('span', attrs={'itemprop':'actors'})
    actor = [a.text.strip() for a in actorSpan]
    #get the summary of the movie
    summary = m_main.find('div', attrs= {'class':'summary_text'}).text.strip()
    
    m_main_bottom = msoup.find('div', attrs={'id':'main_bottom'})
    
   #get the description and description writer
    if m_main_bottom.find('div', attrs={'itemprop':'description'}):
        descriptionDiv = m_main_bottom.find('div', attrs={'itemprop':'description'})
        description = descriptionDiv.p.text.strip()
        if descriptionDiv.a:
            description_writer = descriptionDiv.a.text.strip()
    else:
        description= None
        description_writer = None

    #plot key words
    if m_main_bottom.find('div', attrs={'itemprop':'keywords'}):
        plot_keyword_div = m_main_bottom.find('div', attrs={'itemprop':'keywords'})
        keyword_spanlist = plot_keyword_div.find_all('span', attrs={'itemprop':'keywords'})
        plot_keyword = [a.text for a in keyword_spanlist]
    else:
        plot_keyword = None

    # including information about country, language, date, filming location, color, budget and aspect ratio, box office
    # and so on.
    content_detail = m_main_bottom.find('div',attrs={'id':'titleDetails'})
    #time = content_detail.find('time', attrs= {'itemprop':'duration'}).text
    detail = content_detail.find_all('h4', attrs={'class':'inline'})
    detail_header = [aa.text for aa in detail]
    detail_content = content_detail.find_all('div', attrs={'class':'txt-block'})
    detail_list = [t.text.strip() for t in detail_content]
    t = detail_list
    #time = [t[i] for i,content in enumerate(t) if content.startswith('Runtime')]
    #color = [t[i] for i,content in enumerate(t) if content.startswith('Color')]
    #language = [t[i] for i,content in enumerate(t) if content.startswith('Language')]
    for content in t:
        if content.startswith('Runtime'):
            mtime=content
        
        elif content.startswith('Color'):
            color=content
        
        elif content.startswith('Language'):
            language=content
            
        elif content.startswith('Budget'):
            budget=content
            
        elif 'Facebook' in content:
            a = content_detail.div.a
            facebook_link = a['href'] 
            
        elif content.startswith('Gross'):
            gross=content
        
        elif 'Official site' in detail_list:
            offcialsite = 'True'
        
    if not mtime:
        mtime = None
        
    if not color:
        color = None
        
    if not language:
        language = None 
    
    if not budget:
        budget = None
    
    if not gross:
        gross = None
        
    if not facebook_link:
        facebook_link = None
        
    if not offcialsite:
        offcialsite = None
    
    movie_detial = [title, genre, datepublish, rate, ratingCount, director, writer, actor, summary, description, plot_keyword, mtime, color, language, budget, gross, facebook_link, offcialsite]
    movie_list.append(movie_detial)
    normal_delay = random.normalvariate(1, 0.3)
    time.sleep(normal_delay)

In [5]:
movie_header = ['Title', 'Genre', 'Datepublish', 'Rate', 'RatingCount', 'Director(s)', 'Writer(s)', 'Actor(s)', 'Summary', 'Description', 'PlotKeyword', 'Time', 'Color', 'Language', 'Budget', 'Gross', 'Facebook_link', 'Offcialsite']
df = pd.DataFrame(movie_list, columns=movie_header)

In [6]:
df

Title  \
0                               $5 a Day (2008)   
1                    10 Cloverfield Lane (2016)   
2                               10 Years (2011)   
3                              10,000 BC (2008)   
4                          10,000 Saints (2015)   
5                               100 Feet (2008)   
6                               11-11-11 (2011)   
7                                      11.22.63   
8                              12 Rounds (2009)   
9                              127 Hours (2010)   
10                                    13 (2010)   
11                                  1408 (2007)   
12                             16 Wishes (2010)   
13                              17 Again (2009)   
14                           17 Miracles (2011)   
15                    2 Days in New York (2012)   
16                       2 Days in Paris (2007)   
17                                  2012 (2009)   
18                                    21 (2008)   
19                             21 & Over (2013)   
20                        21 Jump Street (2012)   
21                            27 Dresses (2008)   
22                        28 Weeks Later (2007)   
23                              3 Idiots (2009)   
24            3, 2, 1... Frankie Go Boom (2012)   
25                      30 Days of Night (2007)   
26                    30 Minutes or Less (2011)   
27                300: Rise of an Empire (2014)   
28                                   360 (2011)   
29                          3:10 to Yuma (2007)   
...                                         ...   
2635                                   X (2011)   
2636                                 YPF (2007)   
2637                            Year One (2009)   
2638                     YellowBrickRoad (2010)   
2639                             Yes Man (2008)   
2640                       Yes, Virginia (2009)   
2641                           Yogi Bear (2010)   
2642                           You Again (2010)   
2643              You Can Heal Your Life (2007)   
2644                 You Don't Know Jack (2010)   
2645       You Don't Mess with the Zohan (2008)   
2646                         You Kill Me (2007)   
2647          You May Not Kiss the Bride (2011)   
2648  You Will Meet a Tall Dark Stranger (2010)   
2649                         Young Adult (2011)   
2650                          Young Ones (2014)   
2651                       Your Highness (2011)   
2652                Your Sister's Sister (2011)   
2653                     Youth in Revolt (2009)   
2654          Zack and Miri Make a Porno (2008)   
2655                            Zambezia (2012)   
2656                    Zero Dark Thirty (2012)   
2657                              Zodiac (2007)   
2658                   Zombie Apocalypse (2011)   
2659                    Zombie Strippers (2008)   
2660                          Zombieland (2009)   
2661                          Zombieland (2013)   
2662                           Zookeeper (2011)   
2663                            Zootopia (2016)   
2664                              iSteve (2013)   

                               Genre Datepublish Rate RatingCount  \
0                    [Comedy, Drama]  2009-01-11  6.4       2,826   
1           [Drama, Horror, Mystery]  2016-03-11  7.2     232,334   
2           [Comedy, Drama, Romance]  2012-10-05  6.1      21,235   
3         [Action, Adventure, Drama]  2008-03-07  5.1     116,014   
4             [Comedy, Drama, Music]  2015-08-14  5.9       4,502   
5        [Horror, Mystery, Thriller]  2008-07-24  5.5      11,110   
6        [Horror, Mystery, Thriller]  2011-11-03  4.0      11,471   
7           [Drama, Mystery, Sci-Fi]        None  8.2      52,849   
8           [Action, Crime, Mystery]  2009-03-27  5.6      24,760   
9      [Adventure, Biography, Drama]  2011-01-28  7.6     307,395   
10                 [Drama, Thriller]     2010-03  6.1      37,014   
11                   [Drama, Horror]  2007-06-22  6

In [11]:
df.to_csv('imbd.txt')

In [12]:
df.to_csv('imbd.csv')